# <img src="../../images/logos/python.png" width="23"/> 07.3 The Meaning of Underscore (`_`)
Single and double underscores have a meaning in Python variable and method names. Some of that meaning is merely by convention and intended as a hint to the programmer—and some of it is enforced by the Python interpreter.

- Single Leading Underscore: `_var`
- Single Trailing Underscore: `var_`
- Double Leading Underscore: `__var`
- Double Leading and Trailing Underscore: `__var__`
- Single Underscore: `_`

## <img src="../../images/logos/toc.png" width="20"/> Table of Contents


* [1. Single Leading Underscore: `_var`](#1._single_leading_underscore:_`_var`)
* [2. Single Trailing Underscore: `var_`](#2._single_trailing_underscore:_`var_`)
* [3. Double Leading Underscore: `__var`](#3._double_leading_underscore:_`__var`)
* [4. Double Leading and Trailing Underscore: `__var__`](#4._double_leading_and_trailing_underscore:_`__var__`)
* [5. Single Underscore: `_`](#5._single_underscore:_`_`)
* [Summary](#summary)

---

<a class="anchor" id="1._single_leading_underscore:_`_var`"></a>
## 1. Single Leading Underscore: `_var`

When it comes to variable and method names, the single underscore prefix has a meaning by convention only. It’s a hint to the programmer—and it means what the Python community agrees it should mean, but it does not affect the behavior of your programs.

The underscore prefix is meant as a hint to another programmer that a variable or method starting with a single underscore is intended for internal use. This convention is [defined in PEP 8](http://pep8.org/#descriptive-naming-styles).

This isn’t enforced by Python. Python does not have strong distinctions between “private” and “public” variables like Java does. It’s like someone put up a tiny underscore warning sign that says:

_Hey, this isn’t really meant to be a part of the public interface of this class. Best to leave it alone._

In [11]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 23

In [13]:
t = Test()
t.foo, t._bar

(11, 23)

The single underscore prefix in Python is merely an agreed upon convention—at least when it comes to variable and method names.

However, **leading underscores do impact how names get imported from modules**. Imagine you had the following code in a module called `my_module`:

```python
# This is my_module.py:

def external_func():
    return 23

def _internal_func():
    return 42
```

Now if you use a wildcard import to import all names from the module, Python will not import names with a leading underscore (unless the module defines an `__all__` list that overrides this behavior):

```python
>>> from my_module import *
>>> external_func()
23
>>> _internal_func()
NameError: "name '_internal_func' is not defined"
```

**Note:** wildcard imports should be avoided as they make it unclear which names are present in the namespace. It’s better to stick to regular imports for the sake of clarity.

Unlike wildcard imports, regular imports are not affected by the leading single underscore naming convention:

```python
>>> import my_module
>>> my_module.external_func()
23
>>> my_module._internal_func()
42
```

If you stick to the PEP 8 recommendation that wildcard imports should be avoided, then really all you need to remember is this:

> _Single underscores are a Python naming convention indicating a name is meant for internal use. It is generally not enforced by the Python interpreter and meant as a hint to the programmer only._

<a class="anchor" id="2._single_trailing_underscore:_`var_`"></a>
## 2. Single Trailing Underscore: `var_`

Sometimes the most fitting name for a variable is already taken by a keyword. Therefore names like `class` or def cannot be used as variable names in Python. In this case you can append a single underscore to break the naming conflict:

In [27]:
def make_object(name, class):
    pass

SyntaxError: invalid syntax (295413822.py, line 1)

In [29]:
def make_object(name, class_):
    pass

In summary, a single trailing underscore (postfix) is used by convention to avoid naming conflicts with Python keywords. This convention is [explained in PEP 8](http://pep8.org/#descriptive-naming-styles).

<a class="anchor" id="3._double_leading_underscore:_`__var`"></a>
## 3. Double Leading Underscore: `__var`

The naming patterns covered so far received their meaning from agreed upon conventions only. With Python class attributes (variables and methods) that start with double underscores, things are a little different.

A double underscore prefix causes the Python interpreter to rewrite the attribute name in order to avoid naming conflicts in subclasses. This is also called name _mangling_ —the interpreter changes the name of the variable in a way that makes it harder to create collisions when the class is extended later.

Let’s take a look at the attributes on this object using the [built-in `dir()`](https://docs.python.org/3/library/functions.html#dir) function:

In [48]:
t = Test()
dir(t)

['_Test__baz',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bar',
 'foo']

This gives us a list with the object’s attributes. Let’s take this list and look for our original variable names `foo`, `_bar`, and `__baz`:

- The `self.foo` variable appears unmodified as foo in the attribute list.
- `self._bar` behaves the same way—it shows up on the class as _bar. Like I said before, the leading underscore is just a convention in this case. A hint for the programmer.
- However with `self.__baz`, things look a little different. When you search for `__baz` in that list you’ll see that there is no variable with that name.

If you look closely you’ll see there’s an attribute called ‍`_Test__baz` on this object. This is the _name mangling_ that the Python interpreter applies. It does this to protect the variable from getting overridden in subclasses.

Let’s create another class that **extends** the Test class and attempts to override its existing attributes added in the constructor:

In [91]:
class ExtendedTest(Test):
    def __init__(self):
        super().__init__()
        self.foo = 'overridden'
        self._bar = 'overridden'
        self.__baz = 'overridden'

In [88]:
t2 = ExtendedTest()
t2.foo, t2._bar

('overridden', 'overridden')

In [89]:
t2.__baz

AttributeError: 'ExtendedTest' object has no attribute '__baz'

As you can see `__baz` got turned into `_ExtendedTest__baz` to prevent accidental modification:

In [67]:
t2._ExtendedTest__baz

'overridden'

But the original `_Test__baz` is also still around:

In [68]:
t2._Test__baz

23

Double underscore name mangling is fully transparent to the programmer. Take a look at the following example that will confirm this:

In [25]:
class ManglingTest:
    def __init__(self):
        self.__mangled = 'hello'

    def get_mangled(self):
        return self.__mangled

In [26]:
ManglingTest().get_mangled()

'hello'

In [27]:
ManglingTest().__mangled

AttributeError: 'ManglingTest' object has no attribute '__mangled'

**Does name mangling also apply to method names?** It sure does—name mangling affects all names that start with two underscore characters (“dunders”) in a class context:

In [76]:
class MangledMethod:
    def __method(self):
        return 42

    def call_it(self):
        return self.__method()

In [79]:
MangledMethod().__method()

AttributeError: 'MangledMethod' object has no attribute '__method'

In [78]:
MangledMethod().call_it()

42

<a class="anchor" id="4._double_leading_and_trailing_underscore:_`__var__`"></a>
## 4. Double Leading and Trailing Underscore: `__var__`

Perhaps surprisingly, name mangling is not applied if a name starts and ends with double underscores. Variables surrounded by a double underscore prefix and postfix are left unscathed by the Python interpeter:

In [102]:
class PrefixPostfixTest:
    def __init__(self):
        self.__bam__ = 42

In [103]:
PrefixPostfixTest().__bam__

42

However, names that have both leading and trailing double underscores are reserved for special use in the language. This rule covers things like `__init__` for object constructors, or `__call__` to make an object callable.

<a class="anchor" id="5._single_underscore:_`_`"></a>
## 5. Single Underscore: `_`

Per convention, a single standalone underscore is sometimes used as a name to indicate that a variable is temporary or insignificant.

For example, in the following loop we don’t need access to the running index and we can use `_` to indicate that it is just a temporary value:

In [25]:
for _ in range(5):
    print('Hello, World.')

Hello, World.
Hello, World.
Hello, World.
Hello, World.
Hello, World.


You can also use single underscores in unpacking expressions as a “don’t care” variable to ignore particular values. Again, this meaning is “per convention” only and there’s no special behavior triggered in the Python interpreter. The single underscore is simply a valid variable name that’s sometimes used for this purpose.

In the following code example I’m unpacking a car tuple into separate variables but I’m only interested in the values for color and mileage. However, in order for the unpacking expression to succeed I need to assign all values contained in the tuple to variables. That’s where `_` is useful as a placeholder variable:

In [116]:
car = ('red', 'auto', 12, 3812.4)
color, _, _, mileage = car

In [117]:
color

'red'

In [118]:
mileage

3812.4

In [119]:
_

12

In [120]:
3_000_000_000

3000000000

<a class="anchor" id="summary"></a>
## Summary

|Pattern|Example|Meaning|
|:--|:--|:--|
|Single Leading Underscore|`_var`|Naming convention indicating a name is meant for internal use. Generally not enforced by the Python interpreter (except in wildcard imports) and meant as a hint to the programmer only.|
|Single Trailing Underscore|`var_`|Used by convention to avoid naming conflicts with Python keywords.|
|Double Leading Underscore|`__var`|Triggers name mangling when used in a class context. Enforced by the Python interpreter.|
|Double Leading and Trailing Underscore|`__var__`|Indicates special methods defined by the Python language. Avoid this naming scheme for your own attributes.|
|Single Underscore|`_`|Sometimes used as a name for temporary or insignificant variables (“don’t care”). Also: The result of the last expression in a Python REPL.|

[Reference](https://dbader.org/blog/meaning-of-underscores-in-python)